In [ ]:
from diffusion import DiffusionEquation
import math
import numpy as np

### Simulation Parameters

In [ ]:
# Parameters
library = "../build/libDiffusionEquation.so"
N = 2000
D = 0.1
dt = 0.01
dx = 1.0

In [ ]:
def create_models():
    # Initial concentration
    center = N // 2
    initial_concentration = {(center, center): 1.0}


    # Create the Sequential Diffusion Equation
    sequential = DiffusionEquation(
        library,
        initial_concentration_points=initial_concentration,
        N=N,
        D=D,
        DELTA_T=dt,
        DELTA_X=dx,
    )

    # Create the Parallel Diffusion Equation
    parallel = DiffusionEquation(
        library,
        initial_concentration_points=initial_concentration,
        N=N,
        D=D,
        DELTA_T=dt,
        DELTA_X=dx,
    )
    
    return sequential, parallel

### Verify if the results are the same as the expected results in project description

In [ ]:
sequential, parallel = create_models()
for i in range(500):
    seq_diff = sequential.sequential_step()
    omp_diff = parallel.omp_step()

    if i % 100 == 0:
        print(f"{i}: Sequential: {seq_diff} -- OMP: {omp_diff}")

conc_center_seq = sequential.concentration_matrix[N // 2, N // 2]
conc_center_omp = parallel.concentration_matrix[N // 2, N // 2]
print(f"Concentration at center: {conc_center_seq} -- {conc_center_omp}")

### Verify if the Sequential and OMP solutions are the same

In [ ]:
error = False
sequential, parallel = create_models()
for i in range(500):
    seq_diff = sequential.sequential_step()
    omp_diff = parallel.omp_step()
    
    if i % 100 == 0:
        print(f"Iteration {i}: OMP: {omp_diff}, Sequential: {seq_diff}")
        

    if not math.isclose(seq_diff, omp_diff):
        print(f"Error in iteration {i}: OMP and Sequential results are different")
        print(f"OMP: {omp_diff}, Sequential: {seq_diff}")
        error = True
        
    if np.any(parallel.concentration_matrix != sequential.concentration_matrix):
        print(f"Error in iteration {i}: OMP and Sequential matrix are different")
        error = True

print("All iterations completed successfully")
if not error:
    print("No errors found")